In [3]:
import requests
import openai
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from dotenv import load_dotenv
import os
import time

import ipywidgets as widgets
from IPython.display import display, Image, clear_output

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import pyperclip

from youtube_transcript_api import YouTubeTranscriptApi
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re

from PIL import Image, ImageGrab

load_dotenv()
API_URL = os.getenv("API_URL")
client = openai.Client(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
class YouTube:

    def __init__(self, srcUrl, sectionName):
        self.srcUrl = srcUrl
        self.sectionName = sectionName
        self.videoUrl = self.get_video_url()
        self.html = self.get_html()
        self.videoTitle = self.get_title()
        self.videoTranscription = self.get_video_transcription()
        self.videoText = self.text_from_transcription()

        self.text = self.create_news_text()
        self.title = self.create_news_title()
        self.summary = self.create_news_summary()
        self.imageUrl = ''
        self.imageText = ''
        
        
        
    def get_video_url(self):
        try:
            result = self.srcUrl.replace("/watch%3Fv%3D", "/watch?v=")
            return result

        except Exception as e:
            print(f"Erro ao obter a url do vídeo")
    
    
    
    def get_html(self):
        try:
            browsers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome / 86.0.4240.198Safari / 537.36"}
            page = requests.get(self.url, headers = browsers)
            resposta = page.text
            soup = BeautifulSoup(resposta, 'html.parser')
            return soup
        except:
            return None



    def get_title(self):
        try:
            title = self.html.find('title').get_text()
            return title.strip()
        except Exception as e:
            print(f"Erro ao obter o título")
            return None
    
    
    
    def get_video_transcription(self):
        try:
            video_id = self.url.split("v=")[-1]
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['pt', 'en'])
            transcription = []
            for entry in transcript:
                transcription.append(entry)
            return transcription

        except Exception as e:
            print(f"Erro ao obter a legenda")
    
    
    
    def text_from_transcription(self):
        try:
            text = ''
            for entry in self.transcription:
                text += entry['text']
                text += ' '
            return text

        except Exception as e:
            print(f"Erro ao transformar legenda em texto")
        
        

    def create_news_text(self):
        content = f'Escreva um texto jornalístico que será publicado na seção "{self.sectionName}" de um jornal de grande circulação.\n'
        content += f'Considere como referêcia exclusivamente a transcrição abaixo, extraída de um vídeo de um site de prestígio.\n\n'
        content += f'Título do vídeo: {self.videoTitle}\n\n'
        content += f'{self.videoText}\n\n'
        content += f'O texto da notícia deve possui obrigatoriamente entre 700 e 800 palavras.\n'
        content += f'Se a quantidade de palavras não estiver de acordo com o solicitado, ajuste o campo para atender ao requisito antes de finalizar a resposta.\n'
        content += f'Reforço que deve ser apenas o texto, não incluindo manchete, autor preâmbulo ou qualquer coisa diferente do corpo da notícia.\n'

        message = [{
            'role': 'user',
            'content': content
        }]

        response = client.chat.completions.create(
            messages=message, 
            model="gpt-4o-mini", 
            max_tokens=10000, 
            temperature=0, 
        )
        news_text = response.choices[0].message.content

        return news_text



    def create_news_title(self):
        try:        
            content = f'Escreva um preâmbulo para o texto jornalístico abaixo, que será publicado na seção "{self.sectionName}" de um jornal de grande circulação.\n\n'
            content += f'{self.text}\n\n'
            content += f'Escreva apenas o título da matéria, que deve conter entre 70 e 100 caracteres.\n'
            content += f'Reforço que deve ser apenas o título, não incluindo texto, autor, preâmbulo ou qualquer coisa diferente do título da notícia.\n'
            content += f'Se a quantidade de caracteres não estiver de acordo com o solicitado, ajuste o campo para atender ao requisito antes de finalizar a resposta.'

            message = [{
                'role': 'user',
                'content': content
            }]

            response = client.chat.completions.create(
                messages=message, 
                model="gpt-4o-mini", 
                max_tokens=10000, 
                temperature=0, 
            )
            news_title = response.choices[0].message.content

            return news_title
        except Exception as e:
            print(f'Erro em create_news_title: {e}')
            return 'N/A'



    def create_news_summary(self):
        try:
            content = f'Escreva um título para o texto jornalístico abaixo, que será publicado na seção "{self.sectionName}" de um jornal de grande circulação.\n\n'
            content += f'{self.text}\n\n'
            content += f'Escreva apenas o preâmbulo da matéria, que deve conter entre 190 e 220 caracteres.\n'
            content += f'Reforço que deve ser apenas o preâmbulo, não incluindo texto, autor, título ou qualquer coisa diferente do preâmbulo da notícia.\n'
            content += f'Se a quantidade de caracteres não estiver de acordo com o solicitado, ajuste o campo para atender ao requisito antes de finalizar a resposta.'

            message = [{
                'role': 'user',
                'content': content
            }]

            response = client.chat.completions.create(
                messages=message, 
                model="gpt-4o-mini", 
                max_tokens=10000, 
                temperature=0, 
            )
            news_summary = response.choices[0].message.content

            return news_summary
        except Exception as e:
            print(f'Erro em create_news_summary: {e}')
            return 'N/A'



    def create_news_image(self):
        try:
            teste = input('Insira a imagem na área de transferência e pressione Enter para continuar.')
            image = ImageGrab.grabclipboard()

            if isinstance(image, Image.Image):
                display(image)
            else:
                print("Nenhuma imagem encontrada na área de transferência.")

        except Exception as e:
            print("Problema na seleção da imagem:", e)
            return 'N/A', 'N/A'



    # def save_news(self):
    #     try:
    #         url = f'{API_URL}/api/news'
    #         payload = {
    #             "uid": "johannesadmin",
    #             "supportUid": "johannesadmin",
    #             "h1": self.title,
    #             "h2": self.title,
    #             "summary": self.summary,
    #             "text": self.text,
    #             "imageUrl": self.imageUrl,
    #             "imageText": self.imageText,
    #             "sectionName": self.sectionName,
    #             "__t": "published",
    #         }
    #         response = requests.post(url, json=payload)
    #         if response.status_code != 200:
    #             print(f'Erro: {response.status_code}')
    #     except Exception as e:
    #         print("Problema ao salvar notícia:", e)



